In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [12]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'train',
    target_size=(128,128),
    batch_size=40,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'test',
    target_size=(128,128),
    batch_size=10,
    class_mode='binary')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),     
     
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),   
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [33]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [34]:
DESIRED_ACCURACY = 0.85

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('accuracy')>DESIRED_ACCURACY) and (logs.get('val_accuracy')>DESIRED_ACCURACY )):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [35]:
model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.5327 - accuracy: 0.6250 - val_loss: 1.0054 - val_accuracy: 0.4000
Epoch 2/100
1/1 [==============================] - 1s 980ms/step - loss: 0.4925 - accuracy: 0.7000 - val_loss: 1.0174 - val_accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 1s 996ms/step - loss: 0.3305 - accuracy: 0.8750 - val_loss: 1.2407 - val_accuracy: 0.6000
Epoch 4/100
1/1 [==============================] - 1s 955ms/step - loss: 0.4061 - accuracy: 0.8500 - val_loss: 1.0381 - val_accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.1997 - accuracy: 0.9500 - val_loss: 1.1254 - val_accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 1s 968ms/step - loss: 0.2141 - accuracy: 0.9000 - val_loss: 0.7940 - val_accuracy: 0.8000
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.1232 - accuracy: 1.0000 - val_loss: 0.9382 - val_accuracy: 0.6000
Epoch 8/100
1/1 [=====

Epoch 59/100
1/1 [==============================] - 1s 1s/step - loss: 2.7999e-06 - accuracy: 1.0000 - val_loss: 2.5135 - val_accuracy: 0.7000
Epoch 60/100
1/1 [==============================] - 1s 1s/step - loss: 2.7565e-06 - accuracy: 1.0000 - val_loss: 2.5164 - val_accuracy: 0.7000
Epoch 61/100
1/1 [==============================] - 1s 1s/step - loss: 2.7183e-06 - accuracy: 1.0000 - val_loss: 2.5191 - val_accuracy: 0.7000
Epoch 62/100
1/1 [==============================] - 1s 1s/step - loss: 2.6841e-06 - accuracy: 1.0000 - val_loss: 2.5215 - val_accuracy: 0.7000
Epoch 63/100
1/1 [==============================] - 1s 1s/step - loss: 2.6532e-06 - accuracy: 1.0000 - val_loss: 2.5239 - val_accuracy: 0.7000
Epoch 64/100
1/1 [==============================] - 1s 1s/step - loss: 2.6251e-06 - accuracy: 1.0000 - val_loss: 2.5260 - val_accuracy: 0.7000
Epoch 65/100
1/1 [==============================] - 1s 1s/step - loss: 2.5993e-06 - accuracy: 1.0000 - val_loss: 2.5281 - val_accuracy: 0.7000

In [36]:
model.save('mymodel.h5')